In [3]:
from functools import reduce
import random as random
import numpy as np
from pprint import *


def product(x, y):
    return [xx + [yy] if isinstance(xx, list) else [xx] + [yy] for xx in x for yy in y]


def cartesian_reduct(pools):
    return reduce(lambda x, y: product(x, y), pools)


# setup

cv1 = ['Mon', 'Tu', 'Wed']
cv1_avg_cnt = [12, 132, 21]

cv2 = ['weekend', 'weekday']
pools = [cv1, cv2]

cartesian = cartesian_reduct(pools)
print(cartesian)

example_cv_avg_cnt = []
# for i in range(len(cartesian)):
#    example_cv_avg_cnt.append( random.randrange (1,5,1))
# example_cv_avg_cnt = [3,1,3,2,1,3]

example_cv_avg_cnt = np.array([132, 2123, 4231, 133, 2211, 131])

values = example_cv_avg_cnt
classes = np.full(len(cartesian), np.nan)
print(values)
print(classes)

context_no = len(cartesian)

def mergeClasses(context_idA, context_idB):
    print('merging id {} and id {}'.format(context_idA, context_idB))
    print('old classes = {}'.format(classes))
    new_value = (values[context_idA]+values[context_idB])/2
    # set values of classes A and B to new_value
    idx_a = np.where(classes==context_idA)[0]
    idx_b = np.where(classes==context_idB)[0]
    if idx_b.size == 0: #class is empty, no member yet
        idx_b = context_idB
        
    values[idx_a] = new_value
    values[idx_b] = new_value
    classes[idx_b] = context_idA
    print(idx_b)
    print(classes[idx_b])
    print('new classes = {}'.format(classes))

    
    
def byValueEquality(context_idA, context_idB):
    ret = False
    if(values[context_idA]==values[context_idB]):
        ret = True
    return ret


def zscore(a):
    # @TODO: here a is single index, it should be indexes of all items of class a
    data = np.concatenate((values[:a],values[a+1:]))
    n = len(data)
    mean = sum(data)/n
    variance = sum([((x - mean) ** 2) for x in data]) / n
    stddev = variance ** 0.5
    if stddev == 0: #it's not distribution
        z = np.nan
    else:
        z = (values[a]-mean)/stddev
    return z


def byZscore(context_idA, context_idB):
    ret = False
    
    zscoreA = zscore(context_idA)
    zscoreB = zscore(context_idB)
    
    factor = abs(zscoreA * 0.3)
    abs_result = abs(zscoreA - zscoreB)
    print("Zscore A = {}, B = {}, factor = {}, result = {}".format(zscoreA, zscoreB, factor, abs_result))
    if( abs_result < factor):
        ret = True
    return ret

def compareValues(function_byWhat, context_idA, context_idB):
    ret = False
    print('compare values {} and {}'.format(values[context_idA], values[context_idB]))
    
    if(function_byWhat(context_idA,context_idB)):
        ret = True
        print('same ({} == {})'.format(values[context_idA], values[context_idB]))
        print(classes)
    return ret

def compare(function_byWhat, context_idA, context_idB):
    ret = False
    if(np.isnan(classes[context_idA])):
        classes[context_idA] = context_idA
       
    if(classes[context_idA]==classes[context_idB]):
        ret =  False
    else:
        if compareValues(function_byWhat, context_idA, context_idB):
            mergeClasses(context_idA, context_idB)
            ret =  True
        else:
            ret =  False                    
    
    return ret

for context in range(context_no):
    print("Context {}:", context)
    context_id = context
    
    while context_id < context_no:
        
        context_idA = context
        context_idB = context_id
        
        comparison_result = compare(byZscore, context_idA, context_idB)
        print('id {} id {} comparison result = {}'.format(context_idA, context_idB, comparison_result))
        if comparison_result == False:
            context_id+=1
        else:
            context_id = 0
    print("\n")


[['Mon', 'weekend'], ['Mon', 'weekday'], ['Tu', 'weekend'], ['Tu', 'weekday'], ['Wed', 'weekend'], ['Wed', 'weekday']]
[ 132 2123 4231  133 2211  131]
[nan nan nan nan nan nan]
Context {}: 0
id 0 id 0 comparison result = False
compare values 132 and 2123
Zscore A = -1.066154186379387, B = 0.4598835165976071, factor = 0.3198462559138161, result = 1.5260377029769943
id 0 id 1 comparison result = False
compare values 132 and 4231
Zscore A = -1.066154186379387, B = 3.2937916642492264, factor = 0.3198462559138161, result = 4.3599458506286135
id 0 id 2 comparison result = False
compare values 132 and 133
Zscore A = -1.066154186379387, B = -1.065222956244116, factor = 0.3198462559138161, result = 0.0009312301352710772
same (132 == 133)
[ 0. nan nan nan nan nan]
merging id 0 and id 3
old classes = [ 0. nan nan nan nan nan]
3
0.0
new classes = [ 0. nan nan  0. nan nan]
id 0 id 3 comparison result = True
id 0 id 0 comparison result = False
compare values 132 and 2123
Zscore A = -1.06587542651009

In [5]:
print(classes)
print(values)


[0. 1. 2. 0. 1. 0.]
[ 131 2167 4231  131 2167  131]


In [1]:
def zscore(a):
    # here a is single index, it should be indexes of all items of class a
    data = values
    idx_a = np.where(classes!=classes[a])[0]
    if idx_a.size == 0: #class is empty, no member yet
        idx_a = a
    for idx in idx_a:
        print(idx)
        data = np.concatenate((data[:idx],data[idx+1:]))
    n = len(data)
    print(n)
    print(data)
    mean = sum(data)/n
    variance = sum([((x - mean) ** 2) for x in data]) / n
    stddev = variance ** 0.5
    if stddev == 0: #it's not distribution
        z = np.nan
    else:
        z = (values[a]-mean)/stddev
    return z

In [4]:
example_cv_avg_cnt = np.array([132, 2123, 4231, 133, 2211, 131])

values = example_cv_avg_cnt
classes = np.array([0,1,2,0,2,0])
print(values)
print(values[0], values[5])
print(classes)
print(zscore(0),zscore(5))
# BUG - dla tych samych wartosci (131) tej samej klasy (0) liczy inny zscore (powinien byc ten sam!)


[ 132 2123 4231  133 2211  131]
132 131
[0 1 2 0 2 0]
-1.066154186379387 -1.0670858053270564
